# Does basic house information reflect house's description?

## Data download

In [1]:
import data_downloader
descriptions_list,list_features_all_c = data_downloader.downloader()

C:\Users\facch\Desktop\DS\ADM\Homework4\data_downloader.py:14: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 14 of the file C:\Users\facch\Desktop\DS\ADM\Homework4\data_downloader.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = bs4.BeautifulSoup(requests.get(url).text)
C:\Users\facch\Desktop\DS\ADM\Homework4\data_downloader.py:51: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that cause

IndexError: list index out of range

In [ ]:
import pickle
with open('raw_data.pkl', 'wb') as f:
    pickle.dump((descriptions_list,list_features_all_c), f, pickle.HIGHEST_PROTOCOL)    

## Data preprocessing

In [ ]:
import pickle
with open('raw_data.pkl', 'rb') as f:
    loaded = pickle.load(f)
    list_features = loaded[1]
    list_description = loaded[0]

In [ ]:
import tfIdf_functions
df = tfIdf_functions.tfIdf_calculator(tfIdf_functions.cleaner(list_description))
description_matrix = np.nan_to_num(df.values)
features_matrix = np.array(list_features)

In [ ]:
import pickle
with open('processed_data.pkl', 'wb') as f:
    pickle.dump((description_matrix,features_matrix), f, pickle.HIGHEST_PROTOCOL)

## Clustering

### Clustering preliminaries

#### Principal component analysis

Before we start to build the actual cluster we need to prepare the data. The most important thing is to check how many cluster we should use in our clustering, and to do so we wish to use the elbow method.

First of all let us load our data.

In [8]:
import pickle
with open('processed_data.pkl', 'rb') as f:
    loaded = pickle.load(f)
    features_matrix = loaded[1]
    description_matrix = loaded[0]

In [13]:
print(len(description_matrix[0]))

18468


Unluckly the description matrix has a very huge number of features, therefore the computation will be very slow, in the order of days of computation, to find an optimal cluster.

For solve this problem we will apply a method of dimension reduction to our data to get a more usable amount of data. In particular, we will use the principal component reduction method. It is a widly used method in statistic to reduce the dimension of random variable without losing too much information. In particular we will set a treshold value of $0.95$ for the variance ratio of our data. That is, we will check how much is the variance rate given by our PCA reduction for different dimensions and choose the first we found having a variance ratio of $0.95$. That means that we are losing $5$% of our initial information.

In [2]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

for n in range(1000,len(description_matrix[0]),1000):
    pca = PCA(n_components=n).fit(description_matrix)
    value = np.cumsum(pca.explained_variance_ratio_)
    print(n, value[-1])
    if value[-1] > 0.95:
        break

1000 0.5800906035782916
2000 0.7373358809031546
3000 0.8284489289787839
4000 0.888605939746952
5000 0.9300534448214751
6000 0.9592126712259268


<Figure size 640x480 with 1 Axes>

As we can see we reduced the dimension of our space from $18468$ to only $6000$ losing a $5$% of precision. We now overwrite our matrx with the reducted one.

In [3]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=6000)
description_matrix = pca.fit_transform(description_matrix)

As usual, we will now save our new data for future uses.

In [7]:
import pickle
with open('pca_data.pkl', 'wb') as f:
    pickle.dump((description_matrix,features_matrix), f, pickle.HIGHEST_PROTOCOL)

#### Elbow method

In [1]:
import pickle
with open('processed_data.pkl', 'rb') as f:
    loaded = pickle.load(f)
    features_matrix = loaded[1]
    description_matrix = loaded[0]

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

mms = StandardScaler()
mms.fit(description_matrix)
data_transformed = mms.transform(description_matrix)

Sum_of_squared_distances = []
K = range(100,1000,100)
for k in K:
    print(k)
    km = KMeans(n_clusters=k)
    km = km.fit(data_transformed)
    Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

100
200
300
400
500


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

mms.fit(features_matrix)
data_transformed = mms.transform(features_matrix)

Sum_of_squared_distances = []
K = range(1,30)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(data_transformed)
    Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
import sklearn.cluster
clustered_features = sklearn.cluster.KMeans(n_clusters=7).fit(features_matrix)
clustered_descriptions = sklearn.cluster.KMeans(n_clusters=40).fit(description_matrix)
features_centers = clustered_features.cluster_centers_
descriptions_centers = clustered_descriptions.cluster_centers_

In [ ]:
def all_indices(value, qlist):
    indices = []
    idx = -1
    while True:
        try:
            idx = qlist.index(value, idx+1)
            indices.append(idx)
        except ValueError:
            break
    return set(indices)

In [ ]:
features_cluster_list = []
for i in range(7):
    features_cluster_list.append(all_indices(i,list(clustered_features.labels_)))

In [ ]:
descriptions_cluster_list = []
for i in range(40):
    descriptions_cluster_list.append(all_indices(i,list(clustered_descriptions.labels_)))

In [ ]:
def jacobi_sim(S1,S2):
    intl = len(S1.intersection(S2))
    unil = len(S1.union(S2))
    return intl/unil

In [ ]:
def jacobi_sim_matrix(setlist1, setlist2):
    results_list = []
    for S1 in setlist1:
        l = []
        for S2 in setlist2:
            l.append(jacobi_sim(S1,S2))
        results_list.append(l)
    return results_list

In [ ]:
results_list = jacobi_sim_matrix(features_cluster_list,descriptions_cluster_list)

In [ ]:
pd.DataFrame(results_list)